In [1]:
import os
import math

# Battery Dimer Analysis

This notebook is a short and schematic summary of the analysis performed for my MRS Advances paper, which discusses and compares the formation of oxygen dimers in Li$_{0.5}$MnO$_3$ and Li$_{0.5}$MnO$_3$.

## Oxygen Oxidation

To start our analysis, we take a look at the magnetic moments of the various elements in both structures. We want to compare the magnetic moments on the Mn, Ir and O atoms for the discharged and 75% charged structure. As we have relaxed all bulk structures with HSE06, we can use these results to discuss the change in oxidation state, by analyzing the change in magnetic moments. Here's how we can extract the magnetic moments from the Outcar using pymatgen's vasp I/O:



In [12]:
from pymatgen.io.vasp.outputs import Outcar

out = Outcar("./data/oxidation/Li2MnO3_hse_OUTCAR")
out.magnetization

{'@class': 'Outcar',
 '@module': 'pymatgen.io.vasp.outputs',
 'charge': ({'d': 0.0, 'p': 0.064, 's': 2.02, 'tot': 2.083},
  {'d': 0.0, 'p': 0.054, 's': 2.016, 'tot': 2.07},
  {'d': 0.0, 'p': 0.053, 's': 2.015, 'tot': 2.068},
  {'d': 0.0, 'p': 0.053, 's': 2.015, 'tot': 2.068},
  {'d': 4.764, 'p': 6.458, 's': 0.36, 'tot': 11.582},
  {'d': 4.764, 'p': 6.458, 's': 0.36, 'tot': 11.582},
  {'d': 0.0, 'p': 3.496, 's': 1.56, 'tot': 5.056},
  {'d': 0.0, 'p': 3.496, 's': 1.56, 'tot': 5.056},
  {'d': 0.0, 'p': 3.491, 's': 1.562, 'tot': 5.053},
  {'d': 0.0, 'p': 3.49, 's': 1.563, 'tot': 5.053},
  {'d': 0.0, 'p': 3.491, 's': 1.562, 'tot': 5.053},
  {'d': 0.0, 'p': 3.49, 's': 1.563, 'tot': 5.053}),
 'drift': [[2.1e-05, 0.000222, -0.000343]],
 'efermi': 2.0776,
 'electrostatic_potential': [-52.7257,
  -53.0824,
  -53.1165,
  -53.1166,
  -65.8272,
  -65.8272,
  -64.2597,
  -64.2597,
  -64.276,
  -64.276,
  -64.276,
  -64.276],
 'is_stopped': False,
 'magnetization': ({'d': 0.0, 'p': -0.0, 's': -0.0, '

We can see that the output of the magnetization is given in terms of the various orbitals and the total for each atom in the structure. Let's extract the total for each element and average the absolute value of the magnetic moment:

In [3]:
total_magnetization = [atom['tot'] for atom in out.magnetization]
li2mno3_magmom = dict()
li2mno3_magmom["Mn"] = (sum([abs(mag) for mag in total_magnetization[4:6]]))/2
li2mno3_magmom["O"] = (sum([abs(mag) for mag in total_magnetization[6:]]))/6
print("Li2MnO3: magnetization on Mn = " + str(li2mno3_magmom["Mn"]))
print("Li2MnO3: magnetization on O = " + str(round(li2mno3_magmom["O"], 3)))

Li2MnO3: magnetization on Mn = 3.014
Li2MnO3: magnetization on O = 0.001


This could be done a lot better, really. I could write a little script to just get this automatically. But it'll do for now, considering we have a paper to finish. Let's continue with the other structures:

In [6]:
total_abs_magnetization = [abs(atom['tot']) for atom in Outcar("./data/oxidation/Li0.5MnO3_hse_OUTCAR").magnetization]
li05mno3_magmom = dict()
li05mno3_magmom["Mn"] = (sum(total_abs_magnetization[1:3]))/2
li05mno3_magmom["O"] = (sum(total_abs_magnetization[3:]))/6
print("Li0.5MnO3: magnetization on Mn = " + str(li05mno3_magmom["Mn"]))
print("Li0.5MnO3: magnetization on O = " + str(round(li05mno3_magmom["O"], 3)))

Li0.5MnO3: magnetization on Mn = 2.963
Li0.5MnO3: magnetization on O = 0.45


In [9]:
total_abs_magnetization = [abs(atom['tot']) for atom in Outcar("./data/oxidation/Li2IrO3_hse_OUTCAR").magnetization]
li2iro3_magmom = dict()
li2iro3_magmom["Ir"] = (sum(total_abs_magnetization[4:6]))/2
li2iro3_magmom["O"] = (sum(total_abs_magnetization[6:]))/6
print("Li2IrO3: magnetization on Ir = " + str(li2iro3_magmom["Ir"]))
print("Li2IrO3: magnetization on O = " + str(round(li2iro3_magmom["O"], 3)))

Li2IrO3: magnetization on Ir = 0.738
Li2IrO3: magnetization on O = 0.076


In [10]:
total_abs_magnetization = [abs(atom['tot']) for atom in Outcar("./data/oxidation/Li0.5IrO3_hse_OUTCAR").magnetization]
li05iro3_magmom = dict()
li05iro3_magmom["Ir"] = (sum(total_abs_magnetization[1:3]))/2
li05iro3_magmom["O"] = (sum(total_abs_magnetization[3:]))/6
print("Li0.5MnO3: magnetization on Ir = " + str(li05iro3_magmom["Ir"]))
print("Li0.5MnO3: magnetization on O = " + str(round(li05iro3_magmom["O"], 3)))

Li0.5MnO3: magnetization on Ir = 1.165
Li0.5MnO3: magnetization on O = 0.371


This was probably not the most efficient way of doing this. This task could have been more easily performed by just copying the values into MATLAB and making the figure. But it could serve as a tutorial in the future.

## Oxygen Lattice framework distortion

Next we'll take a look at the distances between the oxygen atoms. One way to do this would be to open the structure files in VESTA, and then click a lot. But clicking is for people that don't know Python. We've already written a script that uses the voronoi decomposition of the structure in order to find the neighbors, and then reconstruct the potential oxygen dimers around a specific site. So let's do that. Starting by loading the fully discharged and 75% charged structures of Li$_2$MnO$_3$:

In [33]:
from pybat.core import LiRichCathode

li2mno3_cat = LiRichCathode.from_file("./data/oxygen_distortion/o3-li2mno3.json")
li05mno3_cat = LiRichCathode.from_file("./data/oxygen_distortion/o1-li0.5mno3.json")
print(li2mno3_cat)
print()
print(li05mno3_cat)

Full Formula (Li4 Mn2 O6)
Reduced Formula: Li2MnO3
abc   :   4.979788   4.979788   5.073692
angles:  80.383533  80.383531 119.882393
Sites (12)
  #    #VESTA  SP            a          b          c    magmom
---  --------  ----  ---------  ---------  ---------  --------
  0         1  Li     0.5        0.5        0            0
  1         2  Li    -0         -0          0.5          0
  2         3  Li     0.661809   0.338191   0.5          0
  3         4  Li     0.338191   0.661809   0.5         -0
  4         5  Mn     0.167033   0.832967   0            3.041
  5         6  Mn     0.832967   0.167033  -0           -3.041
  6         7  O      0.781049   0.781049   0.228685    -0
  7         8  O      0.218951   0.218951   0.771315     0
  8         9  O      0.066414   0.42414    0.224551    -0.002
  9        10  O      0.57586    0.933586   0.775449    -0.002
 10        11  O      0.933586   0.57586    0.775449     0.002
 11        12  O      0.42414    0.066414   0.224551     0.00

These have been fully optimized and re-optimized in order to avoid [Pulay stresses.](http://cms.mpi.univie.ac.at/vasp/guide/node161.html) (Notice how you can add links to the markdown. Easy and useful.) Next, we use some Python magic to find all of the distances for both structures and put then in a nice and sorted `List`:

In [31]:
li2mno3_distances = [li2mno3_cat.distance_matrix[indices] for indices in li2mno3_cat.find_oxygen_dimers(4)]
li2mno3_distances.sort(); li2mno3_distances

[2.574242106443846,
 2.574242123892187,
 2.6030370413569446,
 2.7745158036884083,
 2.7745158036884083,
 2.7745158226175515,
 2.774515822617552,
 2.7795183447383067,
 2.7795183447383067,
 2.790746672654144,
 2.8078407335807345,
 2.807840789092908]

In [32]:
li05mno3_distances = [li05mno3_cat.distance_matrix[indices] for indices in li05mno3_cat.find_oxygen_dimers(4)]
li05mno3_distances.sort(); li05mno3_distances

[2.4765280123668454,
 2.47653474005797,
 2.4765773655518086,
 2.6979991457600354,
 2.6980480180966064,
 2.698055437620388,
 2.7886016981628314,
 2.788601698162832,
 2.78861561199193,
 2.788615611991931,
 2.7886205882155957,
 2.788620588215596]

We can immediately see that the discharged structure has a long and short distance (within a certain margin of error). For the charged structure, the short distance has shortened by about 0.1 angstrom (Oh, yeah, the distances are expressed in angstrom, FYI). Three of the long distances have shortened as well, and I have a suspicion that it are the ones that are slightly longer in the original structure. Let's check this by comparing the distances before I foolishly sorted them:

In [41]:
li2mno3_distances = [li2mno3_cat.distance_matrix[indices] for indices in li2mno3_cat.find_oxygen_dimers(4)]
li05mno3_distances = [li05mno3_cat.distance_matrix[indices] for indices in li05mno3_cat.find_oxygen_dimers(4)]
li2mno3_distances

[2.790746672654144,
 2.7745158036884083,
 2.7795183447383067,
 2.574242123892187,
 2.7745158226175515,
 2.6030370413569446,
 2.807840789092908,
 2.7745158036884083,
 2.574242106443846,
 2.7795183447383067,
 2.774515822617552,
 2.8078407335807345]

In [42]:
[dis1 - dis2 for dis1, dis2 in zip(li05mno3_distances, li2mno3_distances)]

[-0.002126084438548226,
 -0.0764677855918019,
 0.009083353424525153,
 -0.09771411152534171,
 -0.29793845706574285,
 0.09496210440309083,
 -0.01922517710097793,
 0.014099808303522643,
 0.21437848177175,
 -0.3029836046803367,
 0.014085875545279514,
 -0.10978529596034647]

Hmm, some of the distances increase! Either the `find_oxygen_dimers` method somehow does not find two lists that correspond for the two structures, or the distortion is more interesting than just a reduction in the distances for the oxygen pairs. That could be possible, and maybe dependent on the lithium configuration. Let's look into this later.

## Fully charged Li$_2$MnO$_3$

We started our investigation of the oxygen dimers by testing the dimer formation of various dimers in the

## Li configuration analysis

In order to study partially charged structures, such as Li$_{0.5}$MnO$_3$ and Li$_{0.5}$IrO$_3$, a choice must be made for the Lithium configuration. Hence, it is important to investigate the optimal lithium configuration, by comparing the formation energies of the various lithium configurations. 

## Symmetry analysis
In this Jupyter notebook, we'll be detailing the dimer analysis we have performed in order to test the stability of the oxygen frameworks for Li$_{0.5}$MnO$_3$ and Li$_{0.5}$IrO$_3$. What we've done for the structures is start with an analysis of the symmetry of the structure in order to find a list of the inequivalent dimers. The dimers themselves are found by analysing the neighbours around each site, using the `pymatgen.chemenv` module. Let's start by loading our in-house developed `pybat` package and loading the 2x2x2 supercell O1-Li$_{0.5}$MnO$_3$ structure with the 1a cation configuration for the Li atoms:

In [5]:
import os
import pybat.core as pb

structure_filename = os.path.abspath("dimer_analysis/data/O1-Li0.5MnO3-1a_222.json")

cat = pb.LiRichCathode.from_file(structure_filename)

Let's have a look at our cathode structure:

In [6]:
print(cat)

Full Formula (Li8 Mn16 O48)
Reduced Formula: LiMn2O6
abc   :   9.996316   9.996316   8.225567
angles:  90.000000  90.000000 120.000000
Sites (96)
  #  #VESTA    SP           a         b          c    magmom
---  --------  ----  --------  --------  ---------  --------
  0  1         Li    0         0         -0            0.003
  1  2         Li    0         0          0.5          0.003
  2  3         Li    0         0.5       -0            0.003
  3  4         Li    0         0.5        0.5          0.003
  4  5         Li    0.5       0         -0            0.003
  5  6         Li    0.5       0          0.5          0.003
  6  7         Li    0.5       0.5       -0            0.003
  7  8         Li    0.5       0.5        0.5          0.003
  8  -         Vac   1         0          0.25         0
  9  -         Vac   0         0          0.75         0
 10  -         Vac   0         0.5        0.25         0
 11  -         Vac   0         0.5        0.75         0
 12  -         V

Note that the structure also keeps track of the vacancies. These are kept in the structure in order for the Voronoi decomposition of the `chemenv` package to keep functioning properly, as well as making it easier to define a migration of the transition metals. Let's first take a look at how many potential dimers there are in the 2x2x2 unit cell of the structure:

In [7]:
len(cat.find_oxygen_dimers())

438

This takes a little longer, because setting up the Voronoi decomposition for the structure using the `pymatgen.analysis.chemenv` package takes some time. It would take quite some time to calculate the formation energy for all of these potential dimers. Fortunately, we've also developed a method for finding the inequivalent dimers in a structure:

In [8]:
cat.find_noneq_dimers()

[(64, 54), (64, 77), (54, 76), (81, 48), (81, 89), (88, 64)]

You can see that of all the dimers in the structure, there are only 6 inequivalent ones. This makes the analysis of the stability of the oxygen framework somewhat easier. I've built upon this method in combination with the `fireworks` package in order to set up a workflow that immediately calculates the formation energy for all the non-equivalent dimers. I won't be executing this here, as it sends these workflows to the pybat server on mLab (Soon to be Mongo DB Atlas.).

## Results

